In [1]:
import numpy as np
import pandas as pd
import pm4py
import numpy as np
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.statistics.eventually_follows.log import get as efg_get
from pm4py.algo.filtering.dfg import dfg_filtering
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import graphviz
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import *
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import variation
from scipy import stats
import sklearn.preprocessing as sk
from scipy.stats import chi2_contingency
import math
import statistics
import graphviz
from statsmodels.stats import multitest
from statsmodels.stats.contingency_tables import SquareTable as ST
import sys
import sklearn
import scipy
from scipy.stats import shapiro
import os
import json
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from package.process_change_analyzer import ProcessChangeAnalyzer
from package.plotting import plot_cell_correlation

In [76]:
#MIMIC-IV
com_pm = pd.read_csv("Outputs/Kidney_Transformed_Var.csv")
el = pd.read_csv("Outputs/Kidney_Transformed_Var.csv")
con_All = pd.read_csv("Outputs/Change_Detection_Kidney_con_All.csv")
cat_All = pd.read_csv("Outputs/Change_Detection_Kidney_cat_All.csv")
df_cat = pd.read_csv("Outputs/Change_Detection_Kidney_df_cat.csv")
df_con = pd.read_csv("Outputs/Change_Detection_Kidney_df_con.csv")

In [77]:
# manually set inputs for the package
attribute_list_con = con_All["E_At"].unique().tolist()
attribute_list_cat = cat_All["E_At"].unique().tolist()
#specify activity and case id column
activity = "concept:name"
case_id = "case:hadm_id"
time_column = "time:timestamp"
#attribute_list_con = attribute_list_con[3:]
attribute_list_con.append("time:timestamp")
# type of variables to use
continuous_columns = attribute_list_con.copy()

categorical_columns = attribute_list_cat.copy()

In [78]:
analyzer = ProcessChangeAnalyzer(com_pm[com_pm.columns[:-1]], activity, case_id, time_column, continuous_columns, categorical_columns)

In [79]:
df = analyzer.df_with_variants

In [30]:
efg_graph = analyzer.efg.copy()

In [31]:
l = list()
for x in efg_graph:
    if (efg_graph[x] <= 30):
        l.append(x)
for e in l:
    del(efg_graph[e])

In [75]:
for rel in efg_graph:
    print(rel[0], rel[1])

Emergency Department Post-ICU Cardiology
Emergency Department Discharged
Pre-ICU Cardiology Post-ICU Cardiology
Pre-ICU Cardiology Discharged
Emergency Department Post-ICU Medicine
Emergency Department Post-ICU Surgery
Pre-ICU Medicine Post-ICU Cardiology
Pre-ICU Medicine Discharged
Pre-ICU Surgery Post-ICU Surgery
Pre-ICU Surgery Discharged
Pre-ICU Surgery Post-ICU Medicine
Pre-ICU Medicine Post-ICU Medicine


In [36]:
act_1 = "Pre-ICU Medicine"
act_2 = "Post-ICU Medicine"

In [37]:
df_efg = eventually_follow_hadms(analyzer.df_with_variants, act_1, act_2)

In [53]:
df_efg_2 = df_efg.set_index([analyzer.case_id_column, analyzer.activity_column])[analyzer.continuous_columns].diff().reset_index()

In [54]:
df_efg_2['previous_activity'] = df_efg_2[analyzer.activity_column].shift(periods=1)

In [55]:
#delete first row
df_efg_2 = df_efg_2.groupby('case:hadm_id').apply(lambda group: group.iloc[1:])
#df_efg_2.loc[df_efg_2["concept:name"] == "Pre-ICU Medicine", "previous_activity"] = np.nan

In [56]:
df_efg_2['sum_timestamp'] = df_efg_2['time:timestamp'].dt.seconds


In [57]:
df_efg_2

,,case:hadm_id,concept:name,Glucose,Potassium,Sodium,Chloride,Bicarbonate,Creatinine,Anion Gap,Urea Nitrogen,...,PTT,INR(PT),PT,pO2,Calculated Total CO2,pCO2,RDW-SD,time:timestamp,previous_activity,sum_timestamp
case:hadm_id,,,,,,,,,,,,,,,,,,,,,,
20124229.0,1,20124229.0,Post-ICU Medicine,-5.400000,0.220000,-4.800000,-3.800000,2.800000,-2.620000,-3.800000,-15.200000,...,64.583333,0.750000,7.900000,NaN,NaN,NaN,-0.160000,4 days 07:40:07,Pre-ICU Medicine,27607
20140574.0,3,20140574.0,Post-ICU Medicine,-34.666667,-0.542857,0.166667,6.642857,-0.357143,-0.592857,-3.642857,-13.857143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 days 19:59:41,Pre-ICU Medicine,71981
20204877.0,5,20204877.0,Post-ICU Medicine,-8.000000,-0.700000,0.666667,2.666667,-2.666667,0.066667,-0.333333,-4.666667,...,-11.600000,-1.600000,-14.100000,26.0,-10.0,-12.0,NaN,5 days 01:15:29,Pre-ICU Medicine,4529
20206823.0,7,20206823.0,Post-ICU Medicine,-3.955556,0.087889,-0.544444,-1.708889,-1.230000,0.131333,-0.727778,7.734444,...,-23.081714,0.293189,3.319892,-42.0,10.0,25.0,0.751515,22 days 06:08:49,Pre-ICU Medicine,22129
20248637.0,9,20248637.0,Post-ICU Medicine,24.250000,-0.125000,1.250000,1.250000,5.500000,-0.275000,-5.750000,-12.750000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 days 15:45:20,Pre-ICU Medicine,56720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29717305.0,425,29717305.0,Post-ICU Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 14:47:13,Pre-ICU Medicine,53233
29723405.0,427,29723405.0,Post-ICU Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 days 02:58:40,Pre-ICU Medicine,10720
29779283.0,429,29779283.0,Post-ICU Medicine,NaN,NaN,NaN,NaN,NaN,-0.500000,NaN,-10.750000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 16:44:04,Pre-ICU Medicine,60244


In [59]:
df_efg_3 = df_efg_2.groupby([analyzer.activity_column, 'previous_activity']).agg(list)

In [60]:
df_efg_3

,,case:hadm_id,Glucose,Potassium,Sodium,Chloride,Bicarbonate,Creatinine,Anion Gap,Urea Nitrogen,Magnesium,...,pH,PTT,INR(PT),PT,pO2,Calculated Total CO2,pCO2,RDW-SD,time:timestamp,sum_timestamp
concept:name,previous_activity,,,,,,,,,,,,,,,,,,,,,
Post-ICU Medicine,Pre-ICU Medicine,"[20124229.0, 20140574.0, 20204877.0, 20206823....","[-5.3999999999999915, -34.666666666666686, -8....","[0.21999999999999975, -0.5428571428571427, -0....","[-4.800000000000011, 0.1666666666666572, 0.666...","[-3.799999999999997, 6.642857142857139, 2.6666...","[2.799999999999997, -0.35714285714285765, -2.6...","[-2.62, -0.592857142857143, 0.0666666666666664...","[-3.8000000000000007, -3.6428571428571423, -0....","[-15.200000000000003, -13.857142857142847, -4....","[0.2650000000000001, nan, 0.1499999999999999, ...",...,"[nan, nan, -1.0, 0.30666666666666487, nan, nan...","[64.58333333333333, nan, -11.600000000000001, ...","[0.75, nan, -1.5999999999999999, 0.29318918918...","[7.9, nan, -14.100000000000001, 3.319891891891...","[nan, nan, 26.0, -42.0, nan, nan, nan, nan, na...","[nan, nan, -10.0, 10.0, nan, nan, nan, nan, na...","[nan, nan, -12.0, 25.0, nan, nan, nan, nan, na...","[-0.1599999999999966, nan, nan, 0.751515151515...","[4 days 07:40:07, 12 days 19:59:41, 5 days 01:...","[27607, 71981, 4529, 22129, 56720, 77531, 0, 7..."


In [66]:
x = pd.concat([analyzer.continuous_grouped_df, df_efg_3])

In [71]:
x.loc[('Post-ICU Medicine', 'Cardiac ICU')]

case:hadm_id            [20004775.0, 20020317.0, 20072904.0, 20088865....
Glucose                 [46.78571428571428, 0.75, 31.714285714285722, ...
Potassium               [-0.42857142857142794, -0.1999999999999993, -0...
Sodium                  [2.2857142857142776, 4.25, 1.0, -0.16666666666...
Chloride                [-2.428571428571445, 3.75, 1.1666666666666856,...
Bicarbonate             [4.071428571428573, 0.25, 2.5, -1.5, -0.600000...
Creatinine              [-0.3857142857142857, -1.45, 0.316666666666666...
Anion Gap               [0.6428571428571423, 0.25, -2.333333333333334,...
Urea Nitrogen           [-6.714285714285715, -29.0, 0.3333333333333357...
Magnesium               [0.050000000000000044, -0.125, 0.1666666666666...
Phosphate               [-0.3500000000000001, -0.47499999999999964, -0...
Calcium                 [0.8250000000000002, -0.025000000000000355, -0...
Hematocrit              [7.300000000000001, 2.1999999999999993, -1.705...
Hemoglobin              [2.45000000000

In [70]:
x.index

MultiIndex([(        'Cardiac ICU', 'Emergency Department'),
            (        'Cardiac ICU',   'Pre-ICU Cardiology'),
            (        'Cardiac ICU',     'Pre-ICU Medicine'),
            (        'Cardiac ICU',      'Pre-ICU Surgery'),
            (         'Discharged',          'Cardiac ICU'),
            (         'Discharged',          'Medical ICU'),
            (         'Discharged',  'Post-ICU Cardiology'),
            (         'Discharged',    'Post-ICU Medicine'),
            (         'Discharged',     'Post-ICU Surgery'),
            (         'Discharged',         'Surgical ICU'),
            (        'Medical ICU', 'Emergency Department'),
            (        'Medical ICU',   'Pre-ICU Cardiology'),
            (        'Medical ICU',     'Pre-ICU Medicine'),
            (        'Medical ICU',      'Pre-ICU Surgery'),
            ('Post-ICU Cardiology',          'Cardiac ICU'),
            ('Post-ICU Cardiology',          'Medical ICU'),
            ('Post-ICU C

In [34]:
def eventually_follow_hadms(df, act_1, act_2):
    df = df.loc[df[activity].isin([act_1, act_2])]
    l = [] 
    hadms = df[case_id].unique()
    rows_list = []
    for hadm_id in hadms:
        curr_act = ""
        first_row = ""
        df_hadm = df.loc[df[case_id] == hadm_id]
        for index, row in df_hadm.iterrows():
        #first act
            if((row[activity] == act_1) & (curr_act == "")):
                curr_act = row[activity]
                first_row = row
            elif((curr_act != "") & (row[activity] == act_2)):
                rows_list.append(first_row)
                rows_list.append(row)
                curr_act = ""
                
    return pd.DataFrame(rows_list)

In [8]:
continuous_diff_df = analyzer.df_with_variants.set_index([analyzer.case_id_column, analyzer.activity_column])[analyzer.continuous_columns].diff().reset_index()

In [10]:
df

,Unnamed: 0,Unnamed: 0.1,subject_id,case:hadm_id,transfer_id,time:timestamp,outtime,concept:name,Glucose,Abnormal Glucose,...,Abnormal pCO2,RDW-SD,Abnormal RDW-SD,admission_location,insurance,ethnicity,marital_status,discharge_location,age,variant
0,0,0,12527107,20002636.0,34228746,2148-02-21 15:50:00,2148-02-21 19:32:00,Emergency Department,NaN,NaN,...,NaN,NaN,NaN,EMERGENCY ROOM,Other,WHITE,DIVORCED,NaN,48.0,"Emergency Department,Pre-ICU Cardiology,Cardia..."
1,1,1,12527107,20002636.0,39096210,2148-02-21 19:32:00,2148-02-25 15:04:36,Pre-ICU Cardiology,88.000000,normal,...,NaN,43.740000,normal,NaN,NaN,NaN,NaN,NaN,NaN,"Emergency Department,Pre-ICU Cardiology,Cardia..."
2,2,2,12527107,20002636.0,35028853,2148-02-25 15:04:36,2148-02-28 17:57:16,Cardiac ICU,110.600000,abnormal high,...,NaN,42.825000,normal,NaN,NaN,NaN,NaN,NaN,NaN,"Emergency Department,Pre-ICU Cardiology,Cardia..."
3,3,3,12527107,20002636.0,36566819,2148-02-28 17:57:16,2148-03-01 17:28:12,Post-ICU Cardiology,89.500000,normal,...,NaN,43.250000,normal,NaN,NaN,NaN,NaN,NaN,NaN,"Emergency Department,Pre-ICU Cardiology,Cardia..."
4,4,4,12527107,20002636.0,31751552,2148-03-01 17:28:12,2148-03-01 17:28:13,Discharged,89.000000,normal,...,NaN,43.300000,normal,NaN,NaN,NaN,NaN,HOME,NaN,"Emergency Department,Pre-ICU Cardiology,Cardia..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13790,13790,13790,13997063,29995505.0,35238347,2123-03-29 14:58:20,2123-03-29 14:58:21,Discharged,80.000000,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHRONIC/LONG TERM ACUTE CARE,NaN,"Emergency Department,Surgical ICU,Post-ICU Med..."
13791,13791,13791,16968104,29998702.0,36837654,2190-11-13 17:27:00,2190-11-13 22:42:00,Emergency Department,NaN,NaN,...,NaN,NaN,NaN,EMERGENCY ROOM,Other,WHITE,MARRIED,NaN,59.0,"Emergency Department,Medical ICU,Post-ICU Surg..."
13792,13792,13792,16968104,29998702.0,37074618,2190-11-13 22:42:00,2190-11-15 08:21:09,Medical ICU,100.333333,abnormal high,...,NaN,62.600000,abnormal high,NaN,NaN,NaN,NaN,NaN,NaN,"Emergency Department,Medical ICU,Post-ICU Surg..."
13793,13793,13793,16968104,29998702.0,38523336,2190-11-15 08:21:09,2190-11-18 10:17:13,Post-ICU Surgery,94.500000,normal,...,NaN,64.766667,abnormal high,NaN,NaN,NaN,NaN,NaN,NaN,"Emergency Department,Medical ICU,Post-ICU Surg..."


In [9]:
continuous_diff_df

,case:hadm_id,concept:name,Glucose,Potassium,Sodium,Chloride,Bicarbonate,Creatinine,Anion Gap,Urea Nitrogen,...,MCV,pH,PTT,INR(PT),PT,pO2,Calculated Total CO2,pCO2,RDW-SD,time:timestamp
0,20002636.0,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,20002636.0,Pre-ICU Cardiology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 03:42:00
2,20002636.0,Cardiac ICU,22.600000,0.400000,0.600000,0.800000,3.400000,-0.280000,-2.800000,-8.2,...,0.050000,NaN,1.65,-0.10,-0.75,NaN,NaN,NaN,-0.915000,3 days 19:32:36
3,20002636.0,Post-ICU Cardiology,-21.100000,-0.610000,1.600000,-1.300000,4.300000,-0.080000,-2.400000,-6.4,...,1.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.425000,3 days 02:52:40
4,20002636.0,Discharged,-0.500000,0.050000,-1.000000,-1.500000,-0.500000,0.000000,1.000000,0.0,...,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050000,1 days 23:30:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13790,29995505.0,Discharged,-5.600000,0.060000,-4.600000,-3.000000,-0.200000,-0.020000,-1.400000,-2.2,...,-1.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 17:48:54
13791,29998702.0,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24701 days 02:28:40
13792,29998702.0,Medical ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 05:15:00
13793,29998702.0,Post-ICU Surgery,-5.833333,0.266667,1.083333,0.666667,0.166667,-0.191667,0.416667,-10.0,...,0.666667,NaN,-4.10,0.35,3.65,NaN,NaN,NaN,2.166667,1 days 09:39:09


In [12]:
continuous_diff_df['previous_activity'] = continuous_diff_df[analyzer.activity_column].shift(periods=1)

In [13]:
continuous_diff_df

,case:hadm_id,concept:name,Glucose,Potassium,Sodium,Chloride,Bicarbonate,Creatinine,Anion Gap,Urea Nitrogen,...,pH,PTT,INR(PT),PT,pO2,Calculated Total CO2,pCO2,RDW-SD,time:timestamp,previous_activity
0,20002636.0,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,20002636.0,Pre-ICU Cardiology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 03:42:00,Emergency Department
2,20002636.0,Cardiac ICU,22.600000,0.400000,0.600000,0.800000,3.400000,-0.280000,-2.800000,-8.2,...,NaN,1.65,-0.10,-0.75,NaN,NaN,NaN,-0.915000,3 days 19:32:36,Pre-ICU Cardiology
3,20002636.0,Post-ICU Cardiology,-21.100000,-0.610000,1.600000,-1.300000,4.300000,-0.080000,-2.400000,-6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.425000,3 days 02:52:40,Cardiac ICU
4,20002636.0,Discharged,-0.500000,0.050000,-1.000000,-1.500000,-0.500000,0.000000,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050000,1 days 23:30:56,Post-ICU Cardiology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13790,29995505.0,Discharged,-5.600000,0.060000,-4.600000,-3.000000,-0.200000,-0.020000,-1.400000,-2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 17:48:54,Post-ICU Medicine
13791,29998702.0,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24701 days 02:28:40,Discharged
13792,29998702.0,Medical ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 05:15:00,Emergency Department
13793,29998702.0,Post-ICU Surgery,-5.833333,0.266667,1.083333,0.666667,0.166667,-0.191667,0.416667,-10.0,...,NaN,-4.10,0.35,3.65,NaN,NaN,NaN,2.166667,1 days 09:39:09,Medical ICU


In [15]:
continuous_diff_df = continuous_diff_df[continuous_diff_df[analyzer.activity_column] != "Emergency Department"]

In [16]:
continuous_diff_df

,case:hadm_id,concept:name,Glucose,Potassium,Sodium,Chloride,Bicarbonate,Creatinine,Anion Gap,Urea Nitrogen,...,pH,PTT,INR(PT),PT,pO2,Calculated Total CO2,pCO2,RDW-SD,time:timestamp,previous_activity
1,20002636.0,Pre-ICU Cardiology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 03:42:00,Emergency Department
2,20002636.0,Cardiac ICU,22.600000,0.400000,0.600000,0.800000,3.400000,-0.280000,-2.800000,-8.200000,...,NaN,1.65,-0.100000,-0.750000,NaN,NaN,NaN,-0.915000,3 days 19:32:36,Pre-ICU Cardiology
3,20002636.0,Post-ICU Cardiology,-21.100000,-0.610000,1.600000,-1.300000,4.300000,-0.080000,-2.400000,-6.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.425000,3 days 02:52:40,Cardiac ICU
4,20002636.0,Discharged,-0.500000,0.050000,-1.000000,-1.500000,-0.500000,0.000000,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050000,1 days 23:30:56,Post-ICU Cardiology
6,20004775.0,Cardiac ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 05:15:00,Emergency Department
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13789,29995505.0,Post-ICU Medicine,-20.650000,-0.720870,-1.878261,2.217391,-6.800000,-0.193043,2.269565,-7.626087,...,NaN,-5.00,0.033333,0.333333,NaN,NaN,NaN,NaN,19 days 00:46:26,Surgical ICU
13790,29995505.0,Discharged,-5.600000,0.060000,-4.600000,-3.000000,-0.200000,-0.020000,-1.400000,-2.200000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 17:48:54,Post-ICU Medicine
13792,29998702.0,Medical ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 05:15:00,Emergency Department
13793,29998702.0,Post-ICU Surgery,-5.833333,0.266667,1.083333,0.666667,0.166667,-0.191667,0.416667,-10.000000,...,NaN,-4.10,0.350000,3.650000,NaN,NaN,NaN,2.166667,1 days 09:39:09,Medical ICU


In [17]:
continuous_grouped_df = continuous_diff_df.groupby([analyzer.activity_column, 'previous_activity']).agg(list)

In [18]:
continuous_grouped_df

case:hadm_id  \
concept:name        previous_activity                                                         
Cardiac ICU         Emergency Department  [20004775.0, 20009241.0, 20017191.0, 20020317....   
                    Pre-ICU Cardiology    [20002636.0, 20009197.0, 20250332.0, 20399252....   
                    Pre-ICU Medicine      [20035470.0, 20124229.0, 20139992.0, 20248637....   
                    Pre-ICU Surgery       [20072904.0, 20416956.0, 21018237.0, 21030045....   
Discharged          Cardiac ICU           [20022423.0, 20139992.0, 20174883.0, 20204398....   
                    Medical ICU           [20060174.0, 20086226.0, 20088508.0, 20108756....   
                    Post-ICU Cardiology   [20002636.0, 20009197.0, 20009241.0, 20017191....   
                    Post-ICU Medicine     [20004775.0, 20008580.0, 20011640.0, 20015496....   
                    Post-ICU Surgery      [20029639.0, 20042202.0, 20053021.0, 20101799....   
                    Surgical ICU          [20023063.0, 20094915.0, 20137177.0, 20204673....   
Medical ICU         Emergency Department  [20029639.0, 20043724.0, 20053986.0, 20058274....   
                    Pre-ICU Cardiology    [21752523.0, 22355109.0, 23937111.0, 25005083....   
                    Pre-ICU Medicine      [20204877.0, 20206823.0, 20437208.0, 20459901....   
                    Pre-ICU Surgery       [20066510.0, 20145585.0, 20453514.0, 20752001....   
Post-ICU Cardiology Cardiac ICU           [20002636.0, 20009197.0, 20009241.0, 20017191....   
                    Medical ICU           [20043724.0, 20272369.0, 20356048.0, 20360941....   
                    Post-ICU Medicine     [20096259.0, 23504833.0, 25229603.0, 25835454....   
                    Post-ICU Surgery      [20741363.0, 22774707.0, 27307767.0, 28946354....   
                    Surgical ICU          [20449515.0, 21303564.0, 21388055.0, 21703018....   
Post-ICU Medicine   Cardiac ICU           [20004775.0, 20020317.0, 20072904.0, 20088865....   
                    Medical ICU           [20053986.0, 20058274.0, 20059021.0, 20063165....   
                    Post-ICU Cardiology   [20096259.0, 20096259.0, 20399252.0, 21213879....   
                    Post-ICU Surgery      [20730479.0, 21113017.0, 23881692.0, 24309759....   
                    Surgical ICU          [20008580.0, 20011640.0, 20015496.0, 20053996....   
Post-ICU Surgery    Cardiac ICU           [20042202.0, 20215586.0, 20741363.0, 20947621....   
                    Medical ICU           [20029639.0, 20145585.0, 20157959.0, 20215264....   
                    Post-ICU Cardiology                            [21737428.0, 27710348.0]   
                    Post-ICU Medicine      [23201707.0, 25770472.0, 28075599.0, 28188004.0]   
                    Surgical ICU          [20053021.0, 20101799.0, 20121390.0, 20166763....   
Pre-ICU Cardiology  Emergency Department  [20002636.0, 20009197.0, 20250332.0, 20399252....   
                    Pre-ICU Medicine      [20379827.0, 20750393.0, 21273831.0, 21960120....   
                    Pre-ICU Surgery                                            [26341594.0]   
Pre-ICU Medicine    Emergency Department  [20035470.0, 20124229.0, 20139992.0, 20140574....   
                    Pre-ICU Cardiology                                         [22726537.0]   
                    Pre-ICU Surgery                                [23256980.0, 24795621.0]   
Pre-ICU Surgery     Emergency Department  [20053021.0, 20066510.0, 20072904.0, 20145585....   
                    Pre-ICU Medicine                               [24795621.0, 28782684.0]   
Surgical ICU        Emergency Department  [20008580.0, 20011640.0, 20015496.0, 20023063....   
                    Pre-ICU Cardiology    [20379827.0, 21703018.0, 22956986.0, 23393787....   
                    Pre-ICU Medicine      [20140574.0, 20392650.0, 20396229.0, 20405881....   
                    Pre-ICU Surgery       [20053021.0, 20362209.0, 20520808.0, 21672423..

In [92]:
analyzer.prepare_correlation()

In [89]:
analyzer.con_cat

case:hadm_id  \
previous_activity    concept:name                                                             
Cardiac ICU          Discharged           [20022423.0, 20139992.0, 20174883.0, 20204398....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20004775.0, 20020317.0, 20072904.0, 20088865....   
                     Post-ICU Surgery     [20042202.0, 20215586.0, 20741363.0, 20947621....   
Emergency Department Cardiac ICU          [20004775.0, 20009241.0, 20017191.0, 20020317....   
                     Discharged           [20002636.0, 20004775.0, 20008580.0, 20009197....   
                     Medical ICU          [20029639.0, 20043724.0, 20053986.0, 20058274....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20004775.0, 20008580.0, 20011640.0, 20015496....   
                     Post-ICU Surgery     [20029639.0, 20042202.0, 20053021.0, 20101799....   
                     Pre-ICU Cardiology   [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Pre-ICU Medicine     [20035470.0, 20124229.0, 20139992.0, 20140574....   
                     Pre-ICU Surgery      [20053021.0, 20066510.0, 20072904.0, 20145585....   
                     Surgical ICU         [20008580.0, 20011640.0, 20015496.0, 20023063....   
Medical ICU          Discharged           [20060174.0, 20086226.0, 20088508.0, 20108756....   
                     Post-ICU Cardiology  [20043724.0, 20272369.0, 20356048.0, 20360941....   
                     Post-ICU Medicine    [20053986.0, 20058274.0, 20059021.0, 20063165....   
                     Post-ICU Surgery     [20029639.0, 20145585.0, 20157959.0, 20215264....   
Post-ICU Cardiology  Discharged           [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20096259.0, 20096259.0, 20399252.0, 21213879....   
                     Post-ICU Surgery                              [21737428.0, 27710348.0]   
Post-ICU Medicine    Discharged           [20004775.0, 20008580.0, 20011640.0, 20015496....   
                     Post-ICU Cardiology  [20096259.0, 23504833.0, 25229603.0, 25835454....   
                     Post-ICU Surgery      [23201707.0, 25770472.0, 28075599.0, 28188004.0]   
Post-ICU Surgery     Discharged           [20029639.0, 20042202.0, 20053021.0, 20101799....   
                     Post-ICU Cardiology  [20741363.0, 22774707.0, 27307767.0, 28946354....   
                     Post-ICU Medicine    [20730479.0, 21113017.0, 23881692.0, 24309759....   
Pre-ICU Cardiology   Cardiac ICU          [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Discharged           [20002636.0, 20009197.0, 20250332.0, 20379827....   
                     Medical ICU          [21752523.0, 22355109.0, 23937111.0, 25005083....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Pre-ICU Medicine                                          [22726537.0]   
                     Surgical ICU         [20379827.0, 21703018.0, 22956986.0, 23393787....   
Pre-ICU Medicine     Cardiac ICU          [20035470.0, 20124229.0, 20139992.0, 20248637....   
                     Discharged           [20035470.0, 20124229.0, 20139992.0, 20140574....   
                     Medical ICU          [20204877.0, 20206823.0, 20437208.0, 20459901....   
                     Post-ICU Cardiology  [20035470.0, 20750393.0, 21486006.0, 21631400....   
                     Post-ICU Medicine    [20124229.0, 20140574.0, 20204877.0, 20206823....   
                     Pre-ICU Cardiology   [20379827.0, 20750393.0, 21273831.0, 21960120....   
                     Pre-ICU Surgery                               [24795621.0, 28782684.0]   
                     Surgical ICU         [20140574.0, 20392650.0, 20396229.0, 20405881..

In [93]:
analyzer.con_cat

case:hadm_id  \
previous_activity    concept:name                                                             
Cardiac ICU          Discharged           [20022423.0, 20139992.0, 20174883.0, 20204398....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20004775.0, 20020317.0, 20072904.0, 20088865....   
                     Post-ICU Surgery     [20042202.0, 20215586.0, 20741363.0, 20947621....   
Emergency Department Cardiac ICU          [20004775.0, 20009241.0, 20017191.0, 20020317....   
                     Discharged           [20002636.0, 20004775.0, 20008580.0, 20009197....   
                     Medical ICU          [20029639.0, 20043724.0, 20053986.0, 20058274....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20004775.0, 20008580.0, 20011640.0, 20015496....   
                     Post-ICU Surgery     [20029639.0, 20042202.0, 20053021.0, 20101799....   
                     Pre-ICU Cardiology   [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Pre-ICU Medicine     [20035470.0, 20124229.0, 20139992.0, 20140574....   
                     Pre-ICU Surgery      [20053021.0, 20066510.0, 20072904.0, 20145585....   
                     Surgical ICU         [20008580.0, 20011640.0, 20015496.0, 20023063....   
Medical ICU          Discharged           [20060174.0, 20086226.0, 20088508.0, 20108756....   
                     Post-ICU Cardiology  [20043724.0, 20272369.0, 20356048.0, 20360941....   
                     Post-ICU Medicine    [20053986.0, 20058274.0, 20059021.0, 20063165....   
                     Post-ICU Surgery     [20029639.0, 20145585.0, 20157959.0, 20215264....   
Post-ICU Cardiology  Discharged           [20002636.0, 20009197.0, 20009241.0, 20017191....   
                     Post-ICU Medicine    [20096259.0, 20096259.0, 20399252.0, 21213879....   
                     Post-ICU Surgery                              [21737428.0, 27710348.0]   
Post-ICU Medicine    Discharged           [20004775.0, 20008580.0, 20011640.0, 20015496....   
                     Post-ICU Cardiology  [20096259.0, 23504833.0, 25229603.0, 25835454....   
                     Post-ICU Surgery      [23201707.0, 25770472.0, 28075599.0, 28188004.0]   
Post-ICU Surgery     Discharged           [20029639.0, 20042202.0, 20053021.0, 20101799....   
                     Post-ICU Cardiology  [20741363.0, 22774707.0, 27307767.0, 28946354....   
                     Post-ICU Medicine    [20730479.0, 21113017.0, 23881692.0, 24309759....   
Pre-ICU Cardiology   Cardiac ICU          [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Discharged           [20002636.0, 20009197.0, 20250332.0, 20379827....   
                     Medical ICU          [21752523.0, 22355109.0, 23937111.0, 25005083....   
                     Post-ICU Cardiology  [20002636.0, 20009197.0, 20250332.0, 20399252....   
                     Pre-ICU Medicine                                          [22726537.0]   
                     Surgical ICU         [20379827.0, 21703018.0, 22956986.0, 23393787....   
Pre-ICU Medicine     Cardiac ICU          [20035470.0, 20124229.0, 20139992.0, 20248637....   
                     Discharged           [20035470.0, 20124229.0, 20139992.0, 20140574....   
                     Medical ICU          [20204877.0, 20206823.0, 20437208.0, 20459901....   
                     Post-ICU Cardiology  [20035470.0, 20750393.0, 21486006.0, 21631400....   
                     Post-ICU Medicine    [20124229.0, 20140574.0, 20204877.0, 20206823....   
                     Pre-ICU Cardiology   [20379827.0, 20750393.0, 21273831.0, 21960120....   
                     Pre-ICU Surgery                               [24795621.0, 28782684.0]   
                     Surgical ICU         [20140574.0, 20392650.0, 20396229.0, 20405881..